In [ ]:
!pip install s3fs

In [ ]:
import dask
import dask.dataframe as dd
import dask.distributed
import pandas as pd

import warnings
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql import Window
from delta.tables import DeltaTable

warnings.filterwarnings("ignore", category=FutureWarning)

# Create SparkSession
spark = SparkSession.builder.appName("DeltaSession") \
            .config("spark.jars.packages", "io.delta:delta-core_2.12:2.3.0") \
            .config("spark.executor.memory", "16G") \
            .config("spark.driver.memory", "8G") \
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
            .getOrCreate()

In [ ]:
ddf = dd.read_parquet(
    "s3://coiled-datasets/timeseries/20-years/parquet",
    storage_options={"anon": True, "use_ssl": True}
)

In [ ]:
ddf.to_parquet('extract/09df_optimize')

In [ ]:
df = (spark.read
         .format("parquet")
         .load("extract/09df_optimize")
     )

In [ ]:
df.count()

In [ ]:
(df
 .write
 .format("delta")
 .save("extract/09delta")
)

In [ ]:
(df
 .write
 .format("delta")
 .save("extract/09delta2")
)